In [1]:
%%writefile mapper_alpha.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

def is_symbolable(word):
    return (len(word) > 1 and word[0].isalpha() and word[1:].isalnum())

def is_casable(word):
    return (len(word) > 1 and word[0].isupper() and word[1:].islower())

for line in sys.stdin:
    try:
        article_id, content = unicode(line.strip()).split('\t', 1)
        content = re.sub("^\W+|\W+$", "", content, flags=re.UNICODE)
    except ValueError as e:
        continue
        
    words = re.split("\W*\s+\W*", content, flags=re.UNICODE)
    
    for word in words:
        if is_symbolable(word):
            print "%s\t%d\t%d" % (word.lower(), 1, 1 if is_casable(word) else 0)
            
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1

Writing mapper_alpha.py


In [2]:
%%writefile reducer_alpha.py

import sys

reload(sys)
sys.setdefaultencoding('utf-8')

cur_word = None
cur_word_total_count = cur_word_indicative_count = 0

for line in sys.stdin:
    try:
        word, total_count, indicative_count = line.strip().split('\t', 2)
        total_count = int(total_count)
        indicative_count = int(indicative_count)
    except ValueError as e:
        continue
        
    if word != cur_word:
        if cur_word:
            print "%s\t%d\t%d" % (cur_word, cur_word_total_count, cur_word_indicative_count)
        
        cur_word = word
        cur_word_total_count = cur_word_indicative_count = 0
    
    cur_word_total_count += total_count
    cur_word_indicative_count += indicative_count
    
if cur_word:
    print "%s\t%d\t%d" % (cur_word, cur_word_total_count, cur_word_indicative_count)

Writing reducer_alpha.py


In [3]:
%%writefile mapper_filter.py

import sys

reload(sys)
sys.setdefaultencoding('utf-8')

def is_name(indicative, total):
    return ((float(indicative) / float(total)) > 0.995)

for line in sys.stdin:
    try:
        word, total_count, indicative_count = unicode(line.strip()).split('\t', 2)
        total_count = int(total_count)
        indicative_count = int(indicative_count)
    except ValueError as e:
        continue
    
    if is_name(indicative_count, total_count):
        print "%s\t%d" % (word, indicative_count)
        print >> sys.stderr, "reporter:counter:Wiki stats,Names,%d" % 1

Writing mapper_filter.py


In [4]:
%%bash

OUT_DIR="name_count"$(date +"%s%6N")
NUM_REDUCERS=8
LOGS="stderr_logs.txt"

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null
hdfs dfs -rm -r -skipTrash ${LOGS} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming name count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper_alpha.py,reducer_alpha.py \
    -mapper "python mapper_alpha.py" \
    -combiner "python reducer_alpha.py" \
    -reducer "python reducer_alpha.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null
    
OUT_DIR1="name_count_summary"$(date +"%s%6N")
NUM_REDUCERS1=1
LOGS1="stderr_summary_logs.txt"

hdfs dfs -rm -r -skipTrash ${OUT_DIR1} > /dev/null
hdfs dfs -rm -r -skipTrash ${LOGS1} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming name count summary" \
    -D mapreduce.job.reduces=${NUM_REDUCERS1} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options=-k2,2nr \
    -D stream.num.map.output.key.fields=2 \
    -files mapper_filter.py \
    -mapper "python mapper_filter.py" \
    -reducer org.apache.hadoop.mapred.lib.IdentityReducer \
    -input ${OUT_DIR}/part-* \
    -output ${OUT_DIR1} > /dev/null
    
hdfs dfs -cat ${OUT_DIR1}/part-00000 | head -5 | tail -1

french	5742	


rm: `name_count1544553571508371': No such file or directory
rm: `stderr_logs.txt': No such file or directory
18/12/11 18:39:37 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/12/11 18:39:37 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/12/11 18:39:39 INFO mapred.FileInputFormat: Total input files to process : 1
18/12/11 18:39:40 INFO mapreduce.JobSubmitter: number of splits:2
18/12/11 18:39:40 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1544553476445_0001
18/12/11 18:39:40 INFO impl.YarnClientImpl: Submitted application application_1544553476445_0001
18/12/11 18:39:40 INFO mapreduce.Job: The url to track the job: http://7e31da3c8dfb:8088/proxy/application_1544553476445_0001/
18/12/11 18:39:40 INFO mapreduce.Job: Running job: job_1544553476445_0001
18/12/11 18:39:48 INFO mapreduce.Job: Job job_1544553476445_0001 running in uber mode : false
18/12/11 18:39:48 INFO mapreduce.Job:  map 0% reduce 0%
18/12/11 18:40:04 INFO mapre